# Assignment 5

Deadline: 11.06.2025 12:00 CEST

## Task

Develop an investment strategy for the Swiss equity market, backtest it using the provided datasets (`market_data.parquet`, `jkp_data.parquet`, `spi_index.csv`) and analyze its performance by benchmarking it against the SPI index. Work with the existing code infrastructure (`qpmwp-course`) and extend it by implementing any additional components needed for the strategy. Write a report that presents your methodology and the results.

### Coding (15 points)

- Selection:
  Implement selection item builder functions (via `SelectionItemBuilder`) to filter stocks based on specific criteria (e.g., exclude low-quality or high-volatility stocks).

- Optimization Data & Constraints:
  Implement functions to prepare optimization data (via `OptimizationItemBuilder`), including any econometric or machine learning-based predictions. These functions should also define optimization constraints (e.g., stock, sector, or factor exposure limits).

- Optimization Model:
  If you choose to create a custom optimization model, develop a class inheriting from Optimization (similar to `MeanVariance`, `LeastSquares`, or `BlackLitterman`). Your class should include methods set_objective and solve for defining the objective function and solving the optimization problem.

- Machine Learning Prediction:
  Integrate a machine learning model to estimate inputs for the optimization, such as expected returns or risk. This could include regression, classification, or learning-to-rank models. I suggest you to use the provided jkp_data as features, but you may also create your own (e.g., technical indicators computed on the return or price series).

- Simulation:
  Backtest the strategy and simulate portfolio returns. Account for fixed costs (1% per annum) and variable (transaction) costs (0.2% per rebalancing).


### Report (15 points):

Generate an HTML report with the following sections:

- High-level strategy overview: Describe the investment strategy you developed.

- Detailed explanation of the backtesting steps: Offer a more comprehensive breakdown of the backtesting process, including a description of the models implemented (e.g., details of the machine learning method used).

- Backtesting results:
    
    - Charts: Include visual representations (e.g., cumulative performance charts, rolling 3-year returns, etc.).
    - Descriptive statistics: Present key statistics such as mean, standard deviation, drawdown, turnover, and Sharpe ratio (or any other relevant metric) for the full backtest period as well as for subperiods (e.g., the last 5 years, or during bull vs. bear market phases).
    - Compare your strategy against the SPI index.


In [42]:
# Standard library imports
import os
import sys
import copy
from typing import Optional

# Third party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add the project root directory to Python path
project_root = os.path.dirname(os.path.dirname(os.getcwd()))   #<Change this path if needed>
src_path = os.path.join(project_root, 'qpmwp-course\\src')    #<Change this path if needed>
sys.path.append(project_root)
sys.path.append(src_path)

from helper_functions import load_data_msci

In [44]:
# Load data
path_to_data = '../data/'  # <change this to your path to data>

# Load market and jkp data from parquet files
market_data = pd.read_parquet(path = f'{path_to_data}market_data.parquet')
jkp_data = pd.read_parquet(path = f'{path_to_data}jkp_data.parquet')
spi_index = pd.read_csv(f'{path_to_data}spi_index.csv')
data = load_data_msci(path = '../data/', n = 10) # just for testing

In [46]:
data['return_series']

,AT,AU,BE,CA,CH,DE,DK,ES,FI,FR
Index,,,,,,,,,,
1999-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1999-01-04,0.010057,0.009080,0.042147,0.013070,0.035885,0.052249,0.026198,0.069051,0.052778,0.049229
1999-01-05,0.013661,-0.010048,0.020162,0.021940,0.012016,0.001444,-0.001789,0.026011,0.014154,0.011346
1999-01-06,0.000000,0.015264,-0.000078,0.027640,0.015335,0.036205,0.000432,0.000000,0.000000,0.021537
1999-01-07,0.004104,0.016564,-0.016877,-0.003480,-0.011902,-0.020187,-0.019041,-0.015610,0.028012,-0.013856
...,...,...,...,...,...,...,...,...,...,...
2023-04-12,-0.002488,0.004714,-0.012072,0.001708,0.000017,0.002662,0.012029,0.006452,-0.005646,0.000822
2023-04-13,0.001297,-0.003806,-0.000049,0.004728,0.000913,0.002400,0.012177,0.002304,0.000147,0.011425
2023-04-14,0.009533,0.004829,0.007622,0.001950,0.008387,0.005797,0.020079,0.005836,0.003530,0.004589


In [51]:
market_data

,,price,mktcap,liquidity,sector
date,id,,,,
1999-05-06,1,461.000000,1.235952e+09,129080.0,None
1999-05-07,1,455.000000,1.219866e+09,4550.0,None
1999-05-10,1,455.000000,1.219866e+09,910.0,None
1999-05-11,1,460.000000,1.233271e+09,4600.0,None
1999-05-12,1,460.000000,1.233271e+09,460.0,None
...,...,...,...,...,...
2024-04-24,313,9.814924,2.016783e+07,0.0,None
2024-04-25,313,9.814924,2.016783e+07,0.0,None
2024-04-26,313,9.814924,2.016783e+07,0.0,None


In [83]:
# Filtering out rows where the 'id' level is nan
market_data = market_data[market_data.index.get_level_values('id').notna()]
market_data
price_data = market_data['price'].unstack(level='id')
price_data = price_data.sort_index()

# ordering the columns
price_data.columns = price_data.columns.astype(int)
price_data = price_data.reindex(sorted(price_data.columns), axis=1)

# replacing nans with 0s (assuming nan means default)
price_data = price_data.fillna(0)
price_data

id,1,2,3,4,5,6,7,8,9,10,...,304,305,306,307,308,309,310,311,312,313
date,,,,,,,,,,,,,,,,,,,,,
1985-12-31,0.0,0.000000,0.000000,0.0,0.000000,0.000000,145.293401,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1986-01-01,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1986-01-02,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1986-01-03,0.0,0.000000,0.000000,0.0,0.000000,0.000000,153.840072,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1986-01-06,0.0,0.000000,0.000000,0.0,0.000000,0.000000,151.703404,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-24,0.0,435.201781,197.614615,0.0,2422.581977,4332.162077,0.000000,141.090773,0.0,0.0,...,30.228257,18.884686,100.481163,17.476927,57.540770,74.338558,29.503400,18.177592,16.131568,9.814924
2024-04-25,0.0,430.659416,197.029956,0.0,2378.734340,4332.162077,0.000000,140.675800,0.0,0.0,...,30.117935,18.757087,100.481163,17.145087,57.744094,76.013793,28.951465,18.112903,16.152356,9.814924
2024-04-26,0.0,431.957235,194.106663,0.0,2400.658159,4319.713335,0.000000,141.298260,0.0,0.0,...,30.283418,18.884686,100.481163,17.499050,58.150743,76.642006,29.653927,18.695104,16.339449,9.814924


In [86]:
cols_to_keep = price_data.columns[price_data.iloc[-1] != 0.0]
tradable_assets_prices = price_data[cols_to_keep]
tradable_assets_prices


id,2,3,5,6,8,13,15,16,19,22,...,304,305,306,307,308,309,310,311,312,313
date,,,,,,,,,,,,,,,,,,,,,
1985-12-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1986-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1986-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1986-01-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1986-01-06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-24,435.201781,197.614615,2422.581977,4332.162077,141.090773,2.126007,108.523327,23093.748090,524.651023,311.387217,...,30.228257,18.884686,100.481163,17.476927,57.540770,74.338558,29.503400,18.177592,16.131568,9.814924
2024-04-25,430.659416,197.029956,2378.734340,4332.162077,140.675800,2.182593,108.523327,22822.056936,533.251859,309.813230,...,30.117935,18.757087,100.481163,17.145087,57.744094,76.013793,28.951465,18.112903,16.152356,9.814924
2024-04-26,431.957235,194.106663,2400.658159,4319.713335,141.298260,2.126007,112.202084,23148.086321,537.552278,311.124886,...,30.283418,18.884686,100.481163,17.499050,58.150743,76.642006,29.653927,18.695104,16.339449,9.814924
